### Imports

In [1]:
import os
import sys
import yaml
import argparse
import numpy as np
import torch

from hls4ml.utils.config import config_from_pyg_model
from hls4ml.converters import convert_from_pyg_model
# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml/utils'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from config import config_from_pyg_model

# module_path = os.path.abspath(os.path.join('../pyg_to_hls_hls4ml/hls4ml'))
# print(module_path)
# if module_path not in sys.path:
#     sys.path.append(module_path)
# from converters import convert_from_pyg_model


from collections import OrderedDict
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# locals
from utils.models.interaction_network_pyg import InteractionNetwork
from model_wrappers import model_wrapper
from utils.data.dataset_pyg import GraphDataset
from utils.data.fix_graph_size import fix_graph_size

layer: Conv1d
func: <function parse_conv1d_layer at 0x7fb73420e680>
layer: Conv2d
func: <function parse_conv2d_layer at 0x7fb73420e710>
layer: Linear
func: <function parse_linear_layer at 0x7fb73420e8c0>
layer: Softmax
func: <function parse_activation_layer at 0x7fb73420e830>
layer: ReLU
func: <function parse_activation_layer at 0x7fb73420e830>
layer: BatchNorm2d
func: <function parse_batchnorm_layer at 0x7fb73420e7a0>
layer: BatchNorm1d
func: <function parse_batchnorm_layer at 0x7fb73420e7a0>
handler args: ('NodeBlock',)
handler args: ('EdgeBlock',)
handler args: ('EdgeAggregate',)
handler args: ('ResidualBlock',)
handler args: ('NodeEncoder',)
handler args: ('EdgeEncoder',)
layer: NodeBlock
func: <function parse_NodeBlock at 0x7fb73421bef0>
layer: EdgeBlock
func: <function parse_EdgeBlock at 0x7fb7342200e0>
layer: EdgeAggregate
func: <function parse_EdgeAggregate at 0x7fb734220290>
layer: ResidualBlock
func: <function parse_ResidualBlock at 0x7fb734220440>
layer: NodeEncoder
func: <f

### PyTorch Model

In [2]:
torch_model = InteractionNetwork(aggr="add", flow="source_to_target", hidden_size=40)
# torch_model_dict = torch.load("trained_models//IN_pyg_small_add_source_to_target_40_state_dict.pt")
# torch_model.load_state_dict(torch_model_dict)

for name, submodule in torch_model.named_modules():
    if name != "":
        print(f"{name}: {submodule}")

R1: RelationalModel(
  (layers): Sequential(
    (0): Linear(in_features=15, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
R1.layers: Sequential(
  (0): Linear(in_features=15, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)
R1.layers.0: Linear(in_features=15, out_features=40, bias=True)
R1.layers.1: ReLU()
R1.layers.2: Linear(in_features=40, out_features=40, bias=True)
R1.layers.3: ReLU()
R1.layers.4: Linear(in_features=40, out_features=5, bias=True)
O: ObjectModel(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=40, bias=True)
    (1): ReLU()
    (2): Linear(in_features=40, out_features=40, bias=True)
    (3): ReLU()
    (4): Linear(in_features=40, out_features=5, bias=True)
  )
)
O.layers: Sequential(
  (

### HLS Model

In [3]:
# forward_dict: defines the order in which graph-blocks are called in the model's 'forward()' method
forward_dict = OrderedDict()
forward_dict["node_encoder"] = "NodeEncoder"
forward_dict["edge_encoder"] = "EdgeEncoder"
forward_dict["R1"] = "EdgeBlock"
forward_dict["O"] = "NodeBlock"
# forward_dict["res_block"] = "ResidualBlock"
forward_dict["R2"] = "EdgeBlock"

In [4]:
common_dim = 5
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
#         "node_dim": 3,
        "node_attr": 3,
        "node_dim": common_dim,
#         "edge_dim": 4,
        "edge_attr": 4,
    "edge_dim":common_dim
}

In [5]:
# output_dir = "test_GNN"
# config = config_from_pyg_model(torch_model,
#                                    default_precision="ap_fixed<32,12>",
#                                    default_index_precision='ap_uint<16>', 
#                                    default_reuse_factor=1)
# hls_model = convert_from_pyg_model(torch_model,
#                                        n_edge=graph_dims['n_edge'],
#                                        n_node=graph_dims['n_node'],
#                                        edge_attr=graph_dims['edge_attr'],
#                                        node_attr=graph_dims['node_attr'],
#                                        edge_dim=graph_dims['edge_dim'],
#                                        node_dim=graph_dims['node_dim'],
#                                        forward_dictionary=forward_dict, 
#                                        activate_final='sigmoid',
#                                        output_dir=output_dir,
#                                        hls_config=config)

In [6]:
output_dir = "test_GNN"
config = config_from_pyg_model(torch_model,
                                   default_precision="ap_fixed<64,17>",
                                   default_index_precision='ap_uint<16>', 
                                   default_reuse_factor=8)
print(f"config: {config}")
hls_model = convert_from_pyg_model(torch_model,
                                       n_edge=graph_dims['n_edge'],
                                       n_node=graph_dims['n_node'],
                                       edge_attr=graph_dims['edge_attr'],
                                       node_attr=graph_dims['node_attr'],
                                       edge_dim=graph_dims['edge_dim'],
                                       node_dim=graph_dims['node_dim'],
                                       forward_dictionary=forward_dict, 
                                       activate_final='sigmoid',
                                       output_dir=output_dir,
                                       hls_config=config)

config: {'Model': {'Precision': 'ap_fixed<64,17>', 'IndexPrecision': 'ap_uint<16>', 'ReuseFactor': 8, 'Strategy': 'Latency'}}
PygModelReader node_dim: 5
PygModelReader edge_dim: 5
PygModelReader node_attr: 3
PygModelReader edge_attr: 4
node encoder layer_dict['n_in']: 3
node encoder layer_dict['n_out']: 5
edge encoder b4 update_dict["last_edge_update"]: edge_attr
edge encoder after update_dict["last_edge_update"]: layer5_out
EdgeBlock update_dict["last_edge_update"] b4: layer5_out
EdgeBlock update_dict["last_edge_update"] after: layer6_out
type(update_dict): <class 'dict'>
nodeblock layer_dict: {'name': 'O', 'n_node': 28, 'n_edge': 37, 'node_dim': 5, 'edge_dim': 5, 'activate_final': 'false', 'activation': 'linear', 'n_layers': 3, 'out_dim': 5, 'class_name': 'NodeBlock', 'inputs': ['layer4_out', 'layer7_out'], 'outputs': ['layer8_out']}
EdgeBlock update_dict["last_edge_update"] b4: layer6_out
EdgeBlock update_dict["last_edge_update"] after: layer9_out
NodeEncoder name: node_encoder
self

In [7]:
hls_model.compile()

Writing HLS project
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb7110805d0>])
layer name: node_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711080910>])
layer name: edge_attr. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711080890>])
layer name: edge_index. func: None
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711017d50>])
def_cpp: layer4_t layer4_out[N_LAYER_1_4*N_LAYER_2_4]
layer name: node_encoder. func: ['nnet::encoder<input_t, layer4_t, config4>(node_attr, layer4_out, w4, b4);']
layer.get_variables(): odict_values([<hls4ml.model.hls_layers.ArrayVariable object at 0x7fb711017810>])
def_cpp: layer5_t layer5_out[N_LAYER_1_5*N_LAYER_2_5]
layer name: edge_encoder. func: ['nnet::encoder<input2_t, layer5_t, config5>(edge_attr, layer5_out, w5, b5);']
layer.get_variables(): odict

Done
lib_name: firmware/myproject-Ad43B2B3.so


# Evaluation and prediction: hls_model.predict(input)

### Data

In [19]:
class data_wrapper(object):
    def __init__(self, node_attr, edge_attr, edge_index, target):
        self.x = node_attr
        self.edge_attr = edge_attr
        self.edge_index = edge_index.transpose(0,1)

        node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
        node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
        self.hls_data = [node_attr, edge_attr, edge_index]

        self.target = target
        self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs(graph_indir, graph_dims, n_graphs):
    graph_files = np.array(os.listdir(graph_indir))
    graph_files = np.array([os.path.join(graph_indir, graph_file)
                            for graph_file in graph_files])
    n_graphs_total = len(graph_files)
    IDs = np.arange(n_graphs_total)
    print(f"IDS: {IDs}")
    dataset = GraphDataset(graph_files=graph_files[IDs])

    graphs = []
    for data in dataset[:n_graphs]:
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, data.edge_attr, data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
#         if not bad_graph:
#             graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
    print(f"graphs length: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_dims = {
#         "n_node": 28,
#         "n_edge": 37,
#         "node_dim": 3,
#         "edge_dim": 4
#     }
graphs = load_graphs(graph_indir, graph_dims, n_graphs=10)

IDS: [0 1 2 3 4 5 6 7 8 9]
graphs length: 10
writing test bench data for 1st graph


In [20]:
data = graphs[0]
print(f"graphs len: {len(graphs)}")
print(type(data.hls_data))
print(f"data.x shape:{data.x.shape}")
torch_pred = torch_model(data)
torch_pred = torch_pred.detach().cpu().numpy().flatten()
print(f"torch pred shape: {torch_pred.shape}")
hls_pred = hls_model.predict(data.hls_data)
print(f"hls_pred.shape: {hls_pred.shape}")
MSE = mean_squared_error(torch_pred, hls_pred)
print(f"MSE: {MSE}")

print((np.abs(torch_pred- hls_pred)))
print(torch_pred)
print(hls_pred)

graphs len: 10
<class 'list'>
data.x shape:torch.Size([28, 3])
Node Encoder output mean: 0.2295726239681244. std: 0.38199612498283386
Edge Encoder output mean: 0.15614545345306396. std:0.19126424193382263
R1 output mean: -0.007930385880172253, std: 0.10171911865472794
O output mean: 0.003440827364102006, std: 0.07398408651351929
R2 output mean: -0.07871315628290176, std: 0.0004216776869725436
torch pred shape: (37,)
hls_pred.shape: (37,)
MSE: 2.95171371789138e-08
[2.4983287e-04 2.2977591e-04 2.4873018e-04 2.3421645e-04 2.2894144e-04
 2.1168590e-04 2.2783875e-04 2.1636486e-04 2.0387769e-04 1.8793344e-04
 2.0277500e-04 1.9210577e-04 1.0699034e-05 1.0102987e-05 1.4990568e-05
 6.3478947e-06 4.5269728e-05 7.3015690e-06 1.1444092e-05 5.4836273e-06
 1.5050173e-05 4.7147274e-05 1.0046363e-04 1.0386109e-04 9.5069408e-05
 1.0526180e-04 2.7269125e-05 1.7791986e-04 1.8274784e-04 1.7195940e-04
 1.8429756e-04 1.2406707e-04 2.6315451e-04 2.6923418e-04 2.5627017e-04
 2.7239323e-04 1.9186735e-04]
[0.48

In [10]:
def get_meanNstd(data, torch_pred, hls_pred):
#     print(data.x)
    node_mean = torch.mean(data.x)
    print(f"node_mean: {node_mean}")
    node_std = torch.std(data.x)
    print(f"node_std: {node_std}")
    node_abs_max = torch.max(torch.abs(data.x))
    print(f"node_abs_max: {node_abs_max}")
    edge_mean = torch.mean(data.edge_attr)
    print(f"edge_mean: {edge_mean}")
    edge_std = torch.std(data.edge_attr)
    print(f"edge_std: {edge_std}")
    edge_abs_max = torch.max(torch.abs(data.edge_attr))
    print(f"edge_abs_max: {edge_abs_max}")
    torch_pred_mean = np.mean(torch_pred)
    print(f"torch_pred_mean: {torch_pred_mean}")
    torch_pred_std = np.std(torch_pred)
    print(f"torch_pred_std: {torch_pred_std}")
    torch_pred_abs_max = np.max(np.abs(torch_pred))
    print(f"torch_pred_abs_max: {torch_pred_abs_max}")
    hls_pred_mean = np.mean(hls_pred)
    print(f"hls_pred_mean: {hls_pred_mean}")
    hls_pred_std = np.std(hls_pred)
    print(f"hls_pred_std: {hls_pred_std}")
    hls_pred_abs_max = np.max(np.abs(hls_pred))
    print(f"hls_pred_abs_max: {hls_pred_abs_max}")
    diff_mean = np.mean(torch_pred- hls_pred)
    print(f"diff_mean: {diff_mean}")
    diff_std = np.std(torch_pred- hls_pred)
    print(f"diff_std: {diff_std}")
    diff_abs_max = np.max(np.abs(torch_pred- hls_pred))
    print(f"diff_abs_max: {diff_abs_max}")
    return None

In [21]:
MSE_l = []
for data in graphs:
#     print(f"graphs len: {len(graphs)}")
#     print(type(data.hls_data))
#     print(f"data.x shape:{data.x.shape}")
    torch_pred = torch_model(data)
    torch_pred = torch_pred.detach().cpu().numpy().flatten()
#     print(f"torch pred shape: {torch_pred.shape}")
    hls_pred = hls_model.predict(data.hls_data)
#     print(f"hls_pred.shape: {hls_pred.shape}")
    MSE = mean_squared_error(torch_pred, hls_pred)
#     get_meanNstd(data, torch_pred, hls_pred)
    print(f"MSE: {MSE}")
    MSE_l.append(MSE)
#     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
#     print(f"MAPE: {MAPE}")

MSE_l = np.array(MSE_l)
print(f"MSE_l: {MSE_l}")
print(f"overall MSE: {np.mean(MSE_l)}")
print(MSE_l.shape)

Node Encoder output mean: 0.2295726239681244. std: 0.38199612498283386
Edge Encoder output mean: 0.15614545345306396. std:0.19126424193382263
R1 output mean: -0.007930385880172253, std: 0.10171911865472794
O output mean: 0.003440827364102006, std: 0.07398408651351929
R2 output mean: -0.07871315628290176, std: 0.0004216776869725436
MSE: 2.95171371789138e-08
Node Encoder output mean: 0.18455061316490173. std: 0.3637453317642212
Edge Encoder output mean: 0.1262378692626953. std:0.1998356133699417
R1 output mean: -0.01077501755207777, std: 0.10436180979013443
O output mean: -0.013840502128005028, std: 0.0823395699262619
R2 output mean: -0.08093857020139694, std: 0.0013533675810322165
MSE: 5.046616934123449e-06
Node Encoder output mean: 0.1555982381105423. std: 0.3277280926704407
Edge Encoder output mean: 0.12424665689468384. std:0.19797925651073456
R1 output mean: -0.011131918989121914, std: 0.10759347677230835
O output mean: -0.017698654904961586, std: 0.06976039707660675
R2 output mean: 

In [12]:
for data_instance in data.hls_data:
    print(data_instance.shape)
    
print(data.edge_index.shape)

(28, 3)
(37, 4)
(37, 2)
torch.Size([2, 37])


In [13]:
# class data_wrapper_tau3mu:
#     def __init__(self, node_attr, edge_attr, edge_index, target):
#         self.x = node_attr
#         self.edge_attr = edge_attr
#         self.edge_index = edge_index.transpose(0,1)

#         node_attr, edge_attr, edge_index = self.x.detach().cpu().numpy(), self.edge_attr.detach().cpu().numpy(), self.edge_index.transpose(0, 1).detach().cpu().numpy().astype(np.float32)
#         node_attr, edge_attr, edge_index = np.ascontiguousarray(node_attr), np.ascontiguousarray(edge_attr), np.ascontiguousarray(edge_index)
#         self.hls_data = [node_attr, edge_attr, edge_index]

#         self.target = target
#         self.np_target = np.reshape(target.detach().cpu().numpy(), newshape=(target.shape[0],))

def load_graphs_tau3mu(data_loader, graph_dims:dict, n_graphs =1000):
    """
    params:
    dataloader: pyg dataloader with custom Tau3MuDataset as its dataset
    graph_dims: 
        graph_dims.keys = ["n_node": max number of nodes allowed in a graph/batch,
            "n_edge": max number of edges allowed in a graph/batch,
            "node_dim": feature dim of node,
            "edge_dim": feature dim of edge
        ]
    
    """

    graphs = []
    
    i = 0
    for data in data_loader:
        if i >= n_graphs:
            break
        data.edge_index = data.edge_index.t()#transpose the edge_index
        n_edges = data.edge_attr.shape[0]
#         print((data.y.shape))
#         print(n_edges)
        data.y = data.y.expand(1, n_edges).flatten()
#         print(f"data.y.shape: {data.y.shape}")
        node_attr, edge_attr, edge_index, target, bad_graph = fix_graph_size(data.x, 
                                                                             data.edge_attr, 
                                                                             data.edge_index,
                                                                             data.y,
                                                                             n_node_max=graph_dims['n_node'],
                                                                             n_edge_max=graph_dims['n_edge'])
        target = torch.flatten(target)# flatten target to 1d array
        if not bad_graph:
            graphs.append(data_wrapper(node_attr, edge_attr, edge_index, target))
        i +=1
        
    print(f"n_graphs: {len(graphs)}")

    print("writing test bench data for 1st graph")
    data = graphs[0]
    node_attr, edge_attr, edge_index = data.x.detach().cpu().numpy(), data.edge_attr.detach().cpu().numpy(), data.edge_index.transpose(
        0, 1).detach().cpu().numpy().astype(np.int32)
    os.makedirs('tb_data', exist_ok=True)
    input_data = np.concatenate([node_attr.reshape(1, -1), edge_attr.reshape(1, -1), edge_index.reshape(1, -1)], axis=1)
    np.savetxt('tb_data/input_data.dat', input_data, fmt='%f', delimiter=' ')

    return graphs


graph_indir = "trackml_data/processed_plus_pyg_small"
# graph_indir = "/home/swissman777/projects/Tau3MuGNNs/data"
graph_dims = {
        "n_node": 28,
        "n_edge": 37,
        "node_dim": 3,
        "edge_dim": 4
    }


In [14]:
import pickle as pkl
from torch_geometric.loader.dataloader import DataLoader
from utils.dataset import Tau3MuDataset

# with open('./trackml_data/data_loaders.pickle', 'rb') as file:
with open('./trackml_data/data_loaders_batch_size_1.pickle', 'rb') as file:
    data_loaders= pkl.load(file) #, x_dim, edge_attr_dim 

print(data_loaders)
# for stage in data_loaders.keys():
#     for data in data_loaders[stage]:
#         print(data.x.shape)
#         print(data.edge_index.shape)
#         print(data.edge_attr.shape)

# print(graphs)

{'train': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb710fedcd0>, 'valid': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb710ffcd90>, 'test': <torch_geometric.loader.dataloader.DataLoader object at 0x7fb711001bd0>}


In [15]:
from sklearn.preprocessing import StandardScaler
MSE_l = []
stages = ["train", "valid", "test"]
for stage in stages:
    graphs = load_graphs_tau3mu(data_loaders[stage], graph_dims)
    for data in graphs:
        torch_pred = torch_model(data)
        torch_pred = torch_pred.detach().cpu().numpy()
        hls_pred = hls_model.predict(data.hls_data)
        MSE = mean_squared_error(torch_pred, hls_pred)
        print(f"MSE: {MSE}")
    #     get_meanNstd(data,torch_pred, hls_pred)
    #     print(f"MSE: {MSE}")
        MSE_l.append(MSE)
    #     MAPE = mean_absolute_percentage_error(torch_pred, hls_pred)
    #     print(f"MAPE: {MAPE}")
MSE_l = np.array(MSE_l)
print(f"MSE_l :{MSE_l}")
print(f"MSE means: {np.mean(MSE_l)}")

n_graphs: 1000
writing test bench data for 1st graph
Node Encoder output mean: -4.58297872543335. std: 57.71450424194336
Edge Encoder output mean: 0.1311596781015396. std:124.3978271484375
R1 output mean: 0.6294550895690918, std: 8.74984359741211
O output mean: -0.18502576649188995, std: 3.8369781970977783
R2 output mean: -0.4838564693927765, std: 0.48442843556404114
MSE: 0.00449563842266798
Node Encoder output mean: 0.470353364944458. std: 82.9599609375
Edge Encoder output mean: 0.5936118364334106. std:25.49294662475586
R1 output mean: -0.39440983533859253, std: 14.019100189208984
O output mean: -1.4052281379699707, std: 5.644840240478516
R2 output mean: -1.1919137239456177, std: 0.40434250235557556
MSE: 0.0014012649189680815
Node Encoder output mean: 5.643376350402832. std: 68.88517761230469
Edge Encoder output mean: 0.13115696609020233. std:147.9611053466797
R1 output mean: 0.3691343069076538, std: 14.732288360595703
O output mean: -1.5271148681640625, std: 4.842690944671631
R2 outp

MSE: 0.0016903738724067807
Node Encoder output mean: -6.037869453430176. std: 55.70315170288086
Edge Encoder output mean: 0.13115762174129486. std:119.81351470947266
R1 output mean: 1.1099086999893188, std: 7.785805702209473
O output mean: -0.023599734529852867, std: 3.7224745750427246
R2 output mean: -0.5767393112182617, std: 0.41097864508628845
MSE: 0.004644540138542652
Node Encoder output mean: -19.1888370513916. std: 89.86378479003906
Edge Encoder output mean: -1.3389774560928345. std:40.125640869140625
R1 output mean: 2.2749807834625244, std: 7.504888534545898
O output mean: 0.5846853256225586, std: 6.188779354095459
R2 output mean: -0.8322473764419556, std: 0.16773979365825653
MSE: 0.0009304566774517298
Node Encoder output mean: 4.832551002502441. std: 93.10994720458984
Edge Encoder output mean: 1.8793256282806396. std:24.153179168701172
R1 output mean: 0.7980753183364868, std: 9.077019691467285
O output mean: -2.113492012023926, std: 6.251744747161865
R2 output mean: -0.88405323

MSE: 0.007131770718842745
Node Encoder output mean: -14.418296813964844. std: 94.5704345703125
Edge Encoder output mean: -2.8079891204833984. std:22.084186553955078
R1 output mean: 1.6732393503189087, std: 6.535114288330078
O output mean: -0.21697121858596802, std: 6.416487693786621
R2 output mean: -0.856521725654602, std: 0.059266429394483566
MSE: 0.0009089501108974218
Node Encoder output mean: -5.781871795654297. std: 79.01692962646484
Edge Encoder output mean: -2.0851142406463623. std:81.3246841430664
R1 output mean: 1.2622766494750977, std: 8.504837036132812
O output mean: -0.6652968525886536, std: 5.480447292327881
R2 output mean: -0.8413771390914917, std: 0.20619253814220428
MSE: 0.0013108092825859785
Node Encoder output mean: 5.346708297729492. std: 75.92147827148438
Edge Encoder output mean: 3.0404841899871826. std:124.804931640625
R1 output mean: 0.6163033843040466, std: 13.535513877868652
O output mean: -1.6674197912216187, std: 5.232139587402344
R2 output mean: -0.9895251393

Node Encoder output mean: 4.9091925621032715. std: 66.1908187866211
Edge Encoder output mean: 0.778572678565979. std:143.60794067382812
R1 output mean: -0.10259561240673065, std: 15.500627517700195
O output mean: -1.3583898544311523, std: 4.4972662925720215
R2 output mean: -0.9710046648979187, std: 0.6489900946617126
MSE: 0.007444931659847498
Node Encoder output mean: -7.437678813934326. std: 74.7349624633789
Edge Encoder output mean: 0.5852968096733093. std:160.41004943847656
R1 output mean: 1.3671261072158813, std: 10.046467781066895
O output mean: -0.3443678617477417, std: 5.040627956390381
R2 output mean: -0.6087684035301208, std: 0.4252746105194092
MSE: 0.00778920017182827
Node Encoder output mean: 15.493685722351074. std: 83.80670928955078
Edge Encoder output mean: 5.689905166625977. std:154.24803161621094
R1 output mean: 0.14543762803077698, std: 17.354183197021484
O output mean: -2.832521677017212, std: 5.525025367736816
R2 output mean: -1.0922702550888062, std: 0.5867349505424

MSE: 0.0005643292097374797
Node Encoder output mean: 6.407749176025391. std: 45.43859100341797
Edge Encoder output mean: 0.1311582773923874. std:102.34599304199219
R1 output mean: 0.1775457262992859, std: 9.821762084960938
O output mean: -1.129948616027832, std: 3.4176464080810547
R2 output mean: -0.3667583763599396, std: 0.5645291209220886
MSE: 0.004886911250650883
Node Encoder output mean: 0.5514866709709167. std: 123.78376007080078
Edge Encoder output mean: -0.2676335573196411. std:17.421405792236328
R1 output mean: 1.686841368675232, std: 6.4718499183654785
O output mean: -2.8976428508758545, std: 8.310901641845703
R2 output mean: -0.8237272500991821, std: 0.06296522170305252
MSE: 0.0007721934234723449
Node Encoder output mean: 10.327005386352539. std: 94.6752700805664
Edge Encoder output mean: 0.08300937712192535. std:17.31410789489746
R1 output mean: -0.8709654211997986, std: 15.442224502563477
O output mean: -2.7576355934143066, std: 6.1880879402160645
R2 output mean: -1.3049472

...

Node Encoder output mean: -5.1680731773376465. std: 70.29289245605469
Edge Encoder output mean: 0.13115811347961426. std:154.0582275390625
R1 output mean: 1.1416807174682617, std: 11.087890625
O output mean: -0.35441097617149353, std: 4.777719974517822
R2 output mean: -0.7298113703727722, std: 0.4931747019290924
MSE: 0.006992090493440628
Node Encoder output mean: 10.078130722045898. std: 124.00946044921875
Edge Encoder output mean: -1.0443223714828491. std:26.16164779663086
R1 output mean: 1.2867093086242676, std: 8.465720176696777
O output mean: -3.752427101135254, std: 7.914201736450195
R2 output mean: -0.8838092684745789, std: 0.26990804076194763
MSE: 0.0008030287572182715
Node Encoder output mean: 5.040560245513916. std: 59.82133865356445
Edge Encoder output mean: 0.13115794956684113. std:141.01458740234375
R1 output mean: -0.016700228676199913, std: 16.022159576416016
O output mean: -1.1964442729949951, std: 4.164631366729736
R2 output mean: -1.031288743019104, std: 0.652381002902

MSE: 0.001458007376641035
Node Encoder output mean: 9.202332496643066. std: 114.42973327636719
Edge Encoder output mean: 0.11221546679735184. std:25.436742782592773
R1 output mean: 1.5253841876983643, std: 6.174936771392822
O output mean: -3.420776844024658, std: 7.379908084869385
R2 output mean: -0.7546091675758362, std: 0.11010760813951492
MSE: 0.0006809773622080684
Node Encoder output mean: -5.604105472564697. std: 96.81369018554688
Edge Encoder output mean: -0.8873725533485413. std:16.229597091674805
R1 output mean: 1.348090648651123, std: 8.08864688873291
O output mean: -1.227882981300354, std: 6.654337406158447
R2 output mean: -0.9054287075996399, std: 0.20151343941688538
MSE: 0.001032636035233736
Node Encoder output mean: 6.409521579742432. std: 64.33821105957031
Edge Encoder output mean: 0.13115745782852173. std:142.76901245117188
R1 output mean: 0.15844742953777313, std: 14.78299331665039
O output mean: -1.353442907333374, std: 4.424854755401611
R2 output mean: -0.944865882396

MSE: 0.0013956682523712516
Node Encoder output mean: 15.873781204223633. std: 99.76252746582031
Edge Encoder output mean: 0.614262044429779. std:29.787946701049805
R1 output mean: -1.3119122982025146, std: 16.809362411499023
O output mean: -3.3313112258911133, std: 6.324282169342041
R2 output mean: -1.3915514945983887, std: 0.2839193046092987
MSE: 0.0021344954147934914
Node Encoder output mean: -16.8277645111084. std: 77.52519226074219
Edge Encoder output mean: -0.7901357412338257. std:13.796878814697266
R1 output mean: 0.6628167033195496, std: 5.113602638244629
O output mean: 0.4702047109603882, std: 4.986064910888672
R2 output mean: -0.7759538888931274, std: 0.06803163886070251
MSE: 0.0006709361332468688
Node Encoder output mean: 13.008426666259766. std: 94.83423614501953
Edge Encoder output mean: 0.7231450080871582. std:31.56163787841797
R1 output mean: -1.0365957021713257, std: 16.610862731933594
O output mean: -2.984893321990967, std: 6.188060760498047
R2 output mean: -1.360095739

R2 output mean: -0.6804327368736267, std: 0.4853536784648895
MSE: 0.006942769046872854
Node Encoder output mean: 3.24592924118042. std: 93.27275085449219
Edge Encoder output mean: 0.17410196363925934. std:24.08474349975586
R1 output mean: -0.1225416511297226, std: 14.482266426086426
O output mean: -1.9739362001419067, std: 6.422401428222656
R2 output mean: -1.241716980934143, std: 0.39647382497787476
MSE: 0.0014185244217514992
Node Encoder output mean: 6.014782905578613. std: 70.98992156982422
Edge Encoder output mean: 0.13115762174129486. std:155.2786407470703
R1 output mean: 0.26128771901130676, std: 15.089205741882324
O output mean: -1.372167944908142, std: 4.796010494232178
R2 output mean: -0.8878053426742554, std: 0.6591039896011353
MSE: 0.005992907099425793
Node Encoder output mean: 29.67335319519043. std: 116.84412384033203
Edge Encoder output mean: -1.2871425151824951. std:28.763071060180664
R1 output mean: -1.0655791759490967, std: 17.174362182617188
O output mean: -5.50603389

MSE: 0.0017335412558168173
Node Encoder output mean: 17.799118041992188. std: 80.68323516845703
Edge Encoder output mean: 3.0539121627807617. std:25.563474655151367
R1 output mean: -1.5986387729644775, std: 18.348796844482422
O output mean: -3.1152844429016113, std: 5.3902459144592285
R2 output mean: -1.5556002855300903, std: 0.20551325380802155
MSE: 0.0012725212145596743
Node Encoder output mean: -6.516538143157959. std: 47.749488830566406
Edge Encoder output mean: 0.1311589479446411. std:102.3055419921875
R1 output mean: 0.5590248703956604, std: 5.6508660316467285
O output mean: 0.22254805266857147, std: 3.018059730529785
R2 output mean: -0.3594632148742676, std: 0.3806740641593933
MSE: 0.003454301506280899
Node Encoder output mean: 9.554203987121582. std: 112.63139343261719
Edge Encoder output mean: 0.4220329225063324. std:21.108945846557617
R1 output mean: 1.8014036417007446, std: 7.1679158210754395
O output mean: -3.372758150100708, std: 7.405045986175537
R2 output mean: -0.823523

In [16]:
MSE_l.shape

(3000,)

In [17]:
torch_model.state_dict()

OrderedDict([('R1.layers.0.weight',
              tensor([[-0.2538,  0.1386, -0.2459,  0.0432, -0.1325, -0.1588, -0.2042,  0.0966,
                        0.1674, -0.1823,  0.1546, -0.0666, -0.0163,  0.0235, -0.0862],
                      [-0.0900,  0.1414,  0.1040, -0.2356,  0.0520,  0.2389, -0.2221, -0.1938,
                       -0.0350, -0.1863, -0.1852, -0.2055,  0.0357,  0.2060, -0.0510],
                      [-0.0391,  0.0447,  0.1168, -0.1439, -0.2567, -0.1247, -0.2079, -0.1791,
                        0.1861, -0.2454, -0.0700,  0.0754, -0.1143, -0.0855,  0.1400],
                      [-0.1776,  0.0483,  0.0781,  0.1151,  0.2524, -0.2195, -0.0683, -0.1887,
                        0.2208,  0.1440, -0.2153,  0.1516, -0.2304, -0.0295,  0.2118],
                      [ 0.2282, -0.1460,  0.0092,  0.1550, -0.2500, -0.1213, -0.0713, -0.1675,
                       -0.1598, -0.1473,  0.0241,  0.2531,  0.1716,  0.1419, -0.0643],
                      [-0.0337,  0.0474,  0.2516,  0.0